In [5]:
import requests
import pymongo
import json
import sys
import pandas as pd

In [6]:
config = {
    'api_key': 'RIOT-API-KEY',
    'request_regions': ['NA1', 'LA1', 'LA2', 'BR1', 'EUN1', 'EUW1', 'TR1', 'RU', 'JP1', 'KR', 'OC1', 'VN2', 'PH2', 'TH2', 'TW2', 'SG2'],
    'mongo_uri': "mongo-uri"
}
def get_region(server):
    if server in ['NA1', 'LA1', 'LA2', 'BR1']:
        return 'AMERICAS'
    elif server in ['EUN1', 'EUW1', 'TR1', 'RU']:
        return 'EUROPE'
    elif server in ['JP1', 'KR']:
        return 'ASIA'
    elif server in ['OC1', 'VN2', 'PH2', 'TH2', 'TW2', 'SG2']:
        return 'SEA'

In [7]:
def get_puuid(summoner_id, region, api_key):
    url = f'https://{region.lower()}.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}?api_key={api_key}'
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com"
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()['puuid']
    else:
        print('Request error (@get_summoner_information). HTTP code {}: {}'.format(response.status_code, response.json()))
        return None

In [8]:
def get_top_players(api_key, region='VN2', queue='RANKED_SOLO_5x5'):
    assert region in config['request_regions']
    assert queue in ['RANKED_SOLO_5x5', 'RANKED_FLEX_SR', 'RANKED_FLEX_TT']

    total_users_to_insert = list()
    # master, grandmaster and challenger endpoints
    request_urls = [
        f'https://{region.lower()}.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/{queue}?api_key={api_key}',
        f'https://{region.lower()}.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/{queue}?api_key={api_key}',
        f'https://{region.lower()}.api.riotgames.com/lol/league/v4/masterleagues/by-queue/{queue}?api_key={api_key}'
    ]

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com"
    }
    #----------------------
    for x in request_urls:
        response = requests.get(x, headers=headers)
        if response.status_code == 200:
            try:
                print(f"Region: {region} | Tier: {response.json()['tier']} | Queue: {response.json()['queue']} | Total Players: {len(response.json()['entries'])}")
            except KeyError as e:
                pass
            for y in response.json()['entries']:
                try:
                    y['tier'] = response.json()['tier']
                    y['request_region'] = region
                    y['queue'] = queue
                    y['puuid'] = get_puuid(y['summonerId'], region, api_key) # insert their puuid as well.
                    total_users_to_insert.append(y)
                except KeyError as e:
                    pass
        else:
            print('Request error (@get_top_players). HTTP code {}: {}'.format(response.status_code, response.json()))
            continue

        print('Total users obtained in region {} and queue {}: {}'.format(region, queue, len(total_users_to_insert)))
    return total_users_to_insert

In [ ]:
get_top_players(config['api_key'], region='VN2', queue='RANKED_SOLO_5x5')

In [ ]:
client = pymongo.MongoClient(config['mongo_uri'])
db = client.myLOLDB
collection = db['summoner']
try:
    collection.drop()
except:
    pass
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

In [ ]:
for region in config['request_regions']:
    for queue in ['RANKED_SOLO_5x5', 'RANKED_FLEX_SR', 'RANKED_FLEX_TT']:
        for x in get_top_players(config['api_key'], region=region, queue=queue):
            if not collection.find_one({'puuid': x['puuid']}):
                try:
                    collection.insert_one(x)
                    print(f"Inserted {x['puuid']} into the database.")
                except Exception as e:
                    print(f"Error inserting {x['puuid']} into the database.")
                    print(e)
            else: 
                print(f"Player {x['puuid']} existed in database. Updating...")
                try:
                    collection.update_one({'puuid': x['puuid']}, {'$set': x})
                    print(f"Updated {x['puuid']} in the database.")
                except Exception as e:
                    print(f"Error updating {x['puuid']} in the database.")
                    print(e)

In [9]:
def get_matches(puuid, count, request_region, type='ranked', api_key=config['api_key']):
    url = f'https://{get_region(request_region)}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type={type}&start=0&count={count}&api_key={api_key}'
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.7",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com"
    }
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print('Request error (@get_matches). HTTP code {}: {}'.format(response.status_code, response.json()))
        return None

In [10]:
client = pymongo.MongoClient(config['mongo_uri'])
db = client.myLOLDB
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [ ]:
import time

collection = db['match']

# try:
#     collection.drop()
# except:
#     pass

# Loop thru each summoner in the database and get their matches
batch_size = 1000
num_summoners = db['summoner'].count_documents({})

for i in range(0, num_summoners, batch_size):
    for x in db['summoner'].find().skip(i).limit(batch_size):
        history = get_matches(x['puuid'], 40, x['request_region'])
        if history is None:
            continue
        for match in get_matches(x['puuid'], 50, x['request_region']):
            if collection.find_one({'match_id': match}):
                print(f"Match {match} existed in the database.")
                continue
            try:
                collection.insert_one({'match_id': match})
                print(f"Inserted match {match} into the database.")
            except Exception as e:
                print(f"Error inserting match {match} into the database.")
                print(e)
                time.sleep(1)

In [ ]:
collection = db['match']
collection.update_many({}, {'$set': {
    'processed_1v1': False, 
    'processed_5v5': False
    }
})

In [14]:
collection = db['matchups']

def create_matchup_db(match_id, collection, api_key=config['api_key']):
    url = f"https://{get_region(match_id.split('_')[0]).lower()}.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com"
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        response_json = response.json()

        for player in response_json['info']['participants']:
            if player['teamPosition'] not in ['TOP', 'JUNGLE', 'MIDDLE', 'BOTTOM', 'UTILITY']:
                continue
            # Check existence in database
            p_match_id = f"{match_id}_{player['teamPosition']}"
            if not collection.find_one({'p_match_id': p_match_id}):
                try:
                    collection.insert_one({
                        'p_match_id': p_match_id,
                        'data': [{
                            'championName': player['championName'],
                            'goldEarned': player['goldEarned'],
                            'totalMinionsKilled': player['totalMinionsKilled'],
                            'win': player['win'],
                            'kills': player['kills'],
                            "assists": player['assists'],
                            "deaths": player['deaths'],
                            "visionScore": player['visionScore'],
                            "puuid": player['puuid'],
                            "totalDamageDealtToChampions": player['totalDamageDealtToChampions'],
                            'summonerName': player['summonerName'],
                        }],
                        'gameVersion': response_json['info']['gameVersion']
                    })
                except Exception as e:
                    print(f"Error inserting {p_match_id} into the database.")
                    print(e)
            else:
                 # Get the document with the matching 'p_match_id'
                doc = collection.find_one({'p_match_id': p_match_id})

                # Check if a player with the same 'puuid' already exists in the 'data' list
                if not any(player['puuid'] == p['puuid'] for p in doc['data']):
                    try:
                        collection.update_one(
                            {'p_match_id': p_match_id}, 
                            {'$push': {
                                'data': {
                                    'championName': player['championName'],
                                    'goldEarned': player['goldEarned'],
                                    'totalMinionsKilled': player['totalMinionsKilled'],
                                    'win': player['win'],
                                    'kills': player['kills'],
                                    "assists": player['assists'],
                                    "deaths": player['deaths'],
                                    "visionScore": player['visionScore'],
                                    "puuid": player['puuid'],
                                    "totalDamageDealtToChampions": player['totalDamageDealtToChampions'],
                                    'summonerName': player['summonerName'],
                                }
                            }}
                        )
                    except Exception as e:
                        print(f"Error updating {p_match_id} in the database.")
                        print(e)    
    else:
        print('Request error (@get_match_info). HTTP code {}: {}'.format(response.status_code, response.json()))
        return None

In [15]:
for x in db['match'].find():
    create_matchup_db(x['match_id'], db['matchups'])

Request error (@get_match_info). HTTP code 500: {'status': {'message': 'Internal server error', 'status_code': 500}}
Error inserting NA1_5000972629_MIDDLE into the database.
ac-ho3gjd9-shard-00-02.zum5rnc.mongodb.net:27017: (10054, 'WSAECONNRESET') (configured timeouts: connectTimeoutMS: 20000.0ms)


ServerSelectionTimeoutError: SSL handshake failed: ac-ho3gjd9-shard-00-00.zum5rnc.mongodb.net:27017: [('SSL routines', '', 'tlsv1 alert internal error')] (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),SSL handshake failed: ac-ho3gjd9-shard-00-01.zum5rnc.mongodb.net:27017: [('SSL routines', '', 'tlsv1 alert internal error')] (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),SSL handshake failed: ac-ho3gjd9-shard-00-02.zum5rnc.mongodb.net:27017: [('SSL routines', '', 'tlsv1 alert internal error')] (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 66501b6311efc9b3ab10cc41, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('ac-ho3gjd9-shard-00-00.zum5rnc.mongodb.net', 27017) server_type: Unknown, rtt: None, error=AutoReconnect("SSL handshake failed: ac-ho3gjd9-shard-00-00.zum5rnc.mongodb.net:27017: [('SSL routines', '', 'tls